## Read raw data and generate timestamped graphs

In [ ]:
import os
import json
import numpy as np
from matplotlib import pyplot as plt

data_dirname = "/coc/flash5/mpatel377/repos/SLaTe-PRO/data/RoboCraft/rawAnnotated"

list_of_datafiles = json.load(open(os.path.join(data_dirname, "../phase1_train_data.json")))

TIME_START_HHMMSS = "150000"
TIME_END_HHMMSS = "180000"

In [ ]:
import os
import json

def time_hhmmss_to_seconds(time_hhmmss):
    return int(time_hhmmss[:2]) * 3600 + int(time_hhmmss[2:4]) * 60 + int(time_hhmmss[4:6])

def time_seconds_to_hhmmss(time_seconds):
    return f"{time_seconds//3600:02d}:{(time_seconds%3600)//60:02d}:{(time_seconds%3600)%60:02d}"

def lowercase_table_name(table_name):
    return table_name.lower().replace(" ", "")

TIME_START_SECONDS = time_hhmmss_to_seconds(TIME_START_HHMMSS)
TIME_END_SECONDS = time_hhmmss_to_seconds(TIME_END_HHMMSS)

def process_frames(data_raw, start_time_hhmmss):
    start_time_seconds = time_hhmmss_to_seconds(start_time_hhmmss)
    timestamped_scene_graphs = {}
    for frame in data_raw:
        timestamp_relative = int(frame["name"].split("_")[-1].split(".")[0])//1000
        timestamp_absolute = start_time_seconds + timestamp_relative
        if timestamp_absolute > TIME_START_SECONDS and timestamp_absolute < TIME_END_SECONDS:
            timestamped_scene_graphs[timestamp_relative] = sorted(list(set([lab["category"].split("_")[0] for lab in frame["labels"]])))
            if "person" in timestamped_scene_graphs[timestamp_relative]:
                timestamped_scene_graphs[timestamp_relative].remove("person")
                timestamped_scene_graphs[timestamp_relative] += ["person"]*frame["labels"].count("person")
    return timestamped_scene_graphs


In [ ]:
data = {}
for filename in os.listdir(data_dirname):
    if filename.endswith(".json"):
        metadata = filename.split("_export")[0]
        camera = metadata.split("_")[1].split("-")[1]
        date = "-".join(metadata.split("_")[0].split('-')[:3])
        start_time = metadata.split("_")[0].split('-')[-1]
        table_name = lowercase_table_name([entry["table_name"] for entry in list_of_datafiles if entry["date"] == date and entry["camera_id"] == camera][0])
        print("Date: ", date)
        print("Camera: ", camera)
        print("Filename: ", filename)
        print("Table Name: ", table_name)
        if date not in data:
            data[date] = {}
        if table_name not in data[date]:
            data[date][table_name] = process_frames(json.load(open(os.path.join(data_dirname, filename), "r"))["frames"], start_time)
            print("Number of frames: ", len(json.load(open(os.path.join(data_dirname, filename), "r"))["frames"]))
        print()

In [ ]:
## Check if timestamps are consistent across tables within a date
all_timestamps = {}
for date in data:
    all_timestamps[date] = []
    timestamps_per_table = {}
    for table_name in data[date]:
        timestamps_per_table[table_name] = data[date][table_name].keys()
        all_timestamps[date].extend(timestamps_per_table[table_name])
    all_timestamps[date] = list(set(all_timestamps[date]))
    all_timestamps[date].sort()
    for table_name in timestamps_per_table:
        if set(timestamps_per_table[table_name]) != set(all_timestamps[date]):
            print(f"Timestamps are not consistent for {date} {table_name}")
            diff1 = (set(timestamps_per_table[table_name]) - set(all_timestamps[date]))
            print(len(diff1), diff1)

In [ ]:
## Convert scene graphs to have the structure date -> table_name -> timestamp -> scene graph
scene_graphs_full = {}

for date in data:
    scene_graphs_full[date] = {}
    for timestamp in all_timestamps[date]:
        scene_graphs_full[date][timestamp] = {}
        for table_name in data[date]:
            if timestamp in data[date][table_name]:
                scene_graphs_full[date][timestamp][table_name] = data[date][table_name][timestamp]
            else:
                scene_graphs_full[date][timestamp][table_name] = []
        

In [ ]:
from datetime import datetime

datetimestr = datetime.now().strftime("%Y%m%d_%H%M%S")
data_filename_processed = f"/coc/flash5/mpatel377/repos/SLaTe-PRO/data/RoboCraft/processedData_GT_{datetimestr}.json"

json.dump(scene_graphs_full, open(data_filename_processed,'w'), indent=4)

## Read in robot timestamps and filter data

In [ ]:
### Filter scene_graphs_full_egocentric to only have timestamps that the robot visited
scene_graphs_full_egocentric = scene_graphs_full

### scene_graphs_full_egocentric should be keyed by minute not second
scene_graphs_full_egocentric = {date: {second//60: val for second, val in scene_graphs_full_egocentric[date].items()} for date in scene_graphs_full_egocentric}

average_dt = np.mean([np.array(list(scene_graphs_full_egocentric[date].keys()))[1:] - np.array(list(scene_graphs_full_egocentric[date].keys())[:-1]) for date in scene_graphs_full_egocentric])
print("Median dt: ", np.median(average_dt))


## Generate graph sequences in SLaTe-PRO format with constant timestep

In [ ]:
import torch

In [ ]:
objects_masterlist = open("../../object_list.txt", "r").read().splitlines()
tables = [o for o in objects_masterlist if o.startswith("table")]
supplyshelf = [o for o in objects_masterlist if o.startswith("supply")]
surfaces = tables + supplyshelf
room = [o for o in objects_masterlist if o.startswith("room")]
objects = [o for o in objects_masterlist if o not in tables and o not in supplyshelf and o not in room]

print("Tables:", ', '.join(tables))
print("Supply Shelves:", ', '.join(supplyshelf))
print("Rooms:", ', '.join(room))
print("Objects:", ', '.join(objects))

static_graph_edges = torch.zeros(len(objects_masterlist), len(objects_masterlist), dtype=torch.float32)
active_edges_mask = torch.zeros_like(static_graph_edges, dtype=torch.float32)
for surface in tables + supplyshelf:
    static_graph_edges[objects_masterlist.index(surface), objects_masterlist.index("room")] = 1.0
    for object in objects:
        active_edges_mask[objects_masterlist.index(object), objects_masterlist.index(surface)] = 1.0


In [ ]:
dt = 10.0
print("Using dt: ", dt)

common_data = {
    'dt': dt,
    'start_time': 0.0,
    'end_time': 60*3,
    'node_classes': objects_masterlist,
    'node_categories': ['Rooms' if obj in room else 'Furniture' if obj in surfaces else 'placable_objects' for obj in objects_masterlist],
    'edge_keys': ["ON"],
    'static_nodes': [obj for obj in objects_masterlist if obj not in objects],
    'static_node_categories': ["Rooms", "Furniture"],
    'dataset_type': "RoboCraft",
    'n_stations': 5,
    'stations': ['tableA', 'tableB', 'tableC', 'tableD', 'tableE'],
    'dates': list(scene_graphs_full_egocentric.keys())
}
json.dump(common_data, open("common_data.json", "w"), indent=4)

common_edge_data = {}
common_edge_data['home_graph'] = static_graph_edges.clone()
for obj in objects:
    common_edge_data['home_graph'][objects_masterlist.index(obj), objects_masterlist.index('room')] = 1.0
common_edge_data['nonstatic_edges'] = active_edges_mask.clone()
common_edge_data['seen_edges'] = torch.zeros_like(active_edges_mask, dtype=torch.float32)
torch.save(common_edge_data, "common_edge_data.pt")

common_embedding_map = {}


In [ ]:
def convert_to_slatepro_format(sg_egocentric):
    ## sg_egocentric is a dictionary of table_name -> list of objects
    sg_slatepro = torch.zeros(len(objects_masterlist), len(objects_masterlist), dtype=torch.float32)
    persons_per_table = torch.zeros((common_data['n_stations']), dtype=torch.float32)
    for table_name, objects in sg_egocentric.items():
        for object in objects:
            if object == "person":
                persons_per_table[common_data['stations'].index(table_name)] += 1
            else:
                sg_slatepro[objects_masterlist.index(object), objects_masterlist.index(table_name)] = 1.0
    for object in objects:
        sg_slatepro[objects_masterlist.index(object), objects_masterlist.index(supplyshelf[0])] = 1.0
        
    return sg_slatepro, persons_per_table


In [ ]:
import os
os.makedirs("processed_seqLM_coarse/all", exist_ok=True)

for date in scene_graphs_full_egocentric:
    nodes = torch.tensor([i for i in range(len(objects_masterlist))], dtype=torch.float32)
    times = torch.arange(common_data['start_time'], common_data['end_time'], common_data['dt'], dtype=torch.float32)
    scene_graph_list = []
    activity_list = []
    for timestamp_slatepro in times:
        closest_timestamp = min(scene_graphs_full_egocentric[date].keys(), key=lambda x: abs(x - timestamp_slatepro))
        sg, per = convert_to_slatepro_format(scene_graphs_full_egocentric[date][closest_timestamp])
        scene_graph_list.append(sg)
        activity_list.append(per)
    torch.save({
            'nodes': nodes,
            'edges': torch.stack(scene_graph_list, dim=0),
            'times': times,
            'active_edges': active_edges_mask,
            'activity': torch.stack(activity_list, dim=0),
        }, f"processed_seqLM_coarse/all/{date}.pt")
